## Create CONSTANTS

In [1]:
import sys

In [2]:
EPM = {
    'min_util': -sys.float_info.max,
    'min_per': 1,
    'max_per': 6,
    'min_avg': 1,
    'max_avg': 5,
    'top_k' : 10
}

### Show global variable

In [3]:
print(EPM)

{'min_util': -1.7976931348623157e+308, 'min_per': 1, 'max_per': 6, 'min_avg': 1, 'max_avg': 5, 'top_k': 10}


# Reading database

In [4]:
UNIT_UTILITY = {}

DATASET = list()

UNIT_COUNT = {}

with open('data/chess_negative.txt', 'r') as file:
    count = 0
    for line in file:
        temp_struc = line.split(":")
        temp_items = list(map(lambda x: "item" + x, temp_struc[0].split(" ")))
        temp_profits = list(map(int, temp_struc[2].split(" ")))
        temp_transaction = {
            "Tid": "T" + str(count),
            "Item": temp_items,
            "Quantity": [1] * len(temp_profits)
        }

        for i in range(len(temp_items)):
            if temp_items[i] not in UNIT_UTILITY:
                UNIT_UTILITY[temp_items[i]] = 0
                UNIT_COUNT[temp_items[i]] = 0
            UNIT_UTILITY[temp_items[i]] += temp_profits[i]
            UNIT_COUNT[temp_items[i]] += 1
            
        DATASET.append(temp_transaction)
        count += 1
        # if count >= 10:
        #     break
    print(count)
for i in UNIT_UTILITY:
    UNIT_UTILITY[i] /= UNIT_COUNT[i]

3196


In [5]:
print(len(UNIT_UTILITY))

75


## Preparation Procedure

The Transaction Weight Utility (TWU) of item \( i \) is given by:

Def 6
Def 4
\[
{TWU}(i) = \sum_{{X \in T_j \\ T_j \subseteq D}} P_U(T_j)
\]

calculate maxPer(i), avgPer(i)
Def 1, Def 2

rearrange order
Def 7


In [6]:
def create_periodic_high_util_itemset(dataset: list, unit_utility: dict, epm: dict) -> list:
    # for i in dataset:
    #     print(i)
    # calculate TWU of each item in dataset
    twu_dict = dict()
    twu_transaction_dict = dict()
    ps_dict = dict()
    transaction_dict = dict()
    for i in dataset:
        twu_transaction_dict[i['Tid']] = 0
        for j in range(len(i['Item'])):
            if i['Item'][j] not in twu_dict:
                twu_dict[i['Item'][j]] = 0
                transaction_dict[i['Item'][j]] = []
                ps_dict[i['Item'][j]] = []

            twu_transaction_dict[i['Tid']] += i['Quantity'][j] * unit_utility[i['Item'][j]] if unit_utility[i['Item'][j]] > 0 else 0
            transaction_dict[i['Item'][j]].append(int(i['Tid'][1:]))

        for j in range(len(i['Item'])):
            twu_dict[i['Item'][j]] += twu_transaction_dict[i['Tid']]

    # update new minU
    sorted_twu_list = list(sorted(twu_dict.items(), key=lambda item: item[1]))
    epm["min_util"] = sorted_twu_list[epm["top_k"] - 1][1] if epm["top_k"] - 1 < len(sorted_twu_list) else sorted_twu_list[-1][1]

    # print(twu_transaction_dict)
    for i in transaction_dict.keys():
        ps_dict[i].append(transaction_dict[i][0])
        for j in range(1, len(transaction_dict[i])):
            ps_dict[i].append(transaction_dict[i][j] - transaction_dict[i][j - 1])

    temp_dataset = list()
    eucs_dict = dict()
    order_set = set()

    for i in dataset:
        temp_item_quantity = list()
        for j in range(len(i['Item'])):
            avg_item = sum(ps_dict[i['Item'][j]])/len(ps_dict[i['Item'][j]])
            # print((i['Item'][j], i['Quantity'][j], unit_utility[i['Item'][j]], twu_dict[i['Item'][j]]))
            if twu_dict[i['Item'][j]] > epm['min_util'] and max(ps_dict[i['Item'][j]]) <= epm['max_per']\
            and avg_item >= epm['min_avg']:

                temp_item_quantity.append((i['Item'][j], i['Quantity'][j], unit_utility[i['Item'][j]], twu_dict[i['Item'][j]]))
                order_set.add((i['Item'][j], unit_utility[i['Item'][j]], twu_dict[i['Item'][j]]))

        temp_item_quantity.sort(key=lambda x: x[3])
        print(temp_item_quantity)
        temp_item_quantity.sort(key=lambda x: x[2])
        print(temp_item_quantity)
        # print(i)
        temp_dataset.append(
            {
                'Tid': i['Tid'],
                'Item': list(map(lambda x: x[0], temp_item_quantity)),
                'Quantity': list(map(lambda x: x[1], temp_item_quantity))
            }
        )
        # print(i, end='\n\n')
    order_list = list(order_set)
    order_list.sort(key=lambda x:x[2])
    order_list.sort(key=lambda x:x[1])
    order_list = [i[0] for i in order_list]

    for i in temp_dataset:
        for j in range(len(i['Item']) - 1):
          for k in range(j + 1, len(i['Item'])):
            if i['Item'][j] + i['Item'][k] not in eucs_dict:
                eucs_dict[i['Item'][j] + i['Item'][k]] = 0
            eucs_dict[i['Item'][j] + i['Item'][k]] += twu_transaction_dict[i['Tid']]

    return temp_dataset, eucs_dict, order_list

In [7]:
NEW_DATASET, EUCS_LIST, ORDER_LIST = create_periodic_high_util_itemset(dataset=DATASET, unit_utility=UNIT_UTILITY, epm=EPM)

[('item9', 1, 16.416492693110648, 831505.058574053), ('item60', 1, -10.956494125119086, 899911.6254927232), ('item40', 1, -38.31230283911672, 908728.2719571937), ('item52', 1, -11.184929356357928, 913509.9183825937), ('item58', 1, -5.495148669796557, 917518.4049957894)]
[('item40', 1, -38.31230283911672, 908728.2719571937), ('item52', 1, -11.184929356357928, 913509.9183825937), ('item60', 1, -10.956494125119086, 899911.6254927232), ('item58', 1, -5.495148669796557, 917518.4049957894), ('item9', 1, 16.416492693110648, 831505.058574053)]
[('item9', 1, 16.416492693110648, 831505.058574053), ('item60', 1, -10.956494125119086, 899911.6254927232), ('item40', 1, -38.31230283911672, 908728.2719571937), ('item52', 1, -11.184929356357928, 913509.9183825937), ('item58', 1, -5.495148669796557, 917518.4049957894)]
[('item40', 1, -38.31230283911672, 908728.2719571937), ('item52', 1, -11.184929356357928, 913509.9183825937), ('item60', 1, -10.956494125119086, 899911.6254927232), ('item58', 1, -5.49514

In [8]:
print(EPM)
EPM = {
    'min_util': -sys.float_info.max,
    'min_per': 1,
    'max_per': 6,
    'min_avg': 1,
    'max_avg': 5,
    'top_k' : 10
}
print(EPM)

{'min_util': 53049.90459736614, 'min_per': 1, 'max_per': 6, 'min_avg': 1, 'max_avg': 5, 'top_k': 10}
{'min_util': -1.7976931348623157e+308, 'min_per': 1, 'max_per': 6, 'min_avg': 1, 'max_avg': 5, 'top_k': 10}


In [9]:
for i in NEW_DATASET:
    print(i)

{'Tid': 'T0', 'Item': ['item40', 'item52', 'item60', 'item58', 'item9'], 'Quantity': [1, 1, 1, 1, 1]}
{'Tid': 'T1', 'Item': ['item40', 'item52', 'item60', 'item58', 'item9'], 'Quantity': [1, 1, 1, 1, 1]}
{'Tid': 'T2', 'Item': ['item40', 'item52', 'item60', 'item58', 'item9'], 'Quantity': [1, 1, 1, 1, 1]}
{'Tid': 'T3', 'Item': ['item40', 'item52', 'item60', 'item58', 'item9'], 'Quantity': [1, 1, 1, 1, 1]}
{'Tid': 'T4', 'Item': ['item40', 'item52', 'item60', 'item58', 'item9'], 'Quantity': [1, 1, 1, 1, 1]}
{'Tid': 'T5', 'Item': ['item40', 'item52', 'item60', 'item58', 'item9'], 'Quantity': [1, 1, 1, 1, 1]}
{'Tid': 'T6', 'Item': ['item40', 'item52', 'item60', 'item58', 'item9'], 'Quantity': [1, 1, 1, 1, 1]}
{'Tid': 'T7', 'Item': ['item40', 'item52', 'item60', 'item58', 'item9'], 'Quantity': [1, 1, 1, 1, 1]}
{'Tid': 'T8', 'Item': ['item40', 'item52', 'item60', 'item58', 'item9'], 'Quantity': [1, 1, 1, 1, 1]}
{'Tid': 'T9', 'Item': ['item40', 'item52', 'item60', 'item58', 'item9'], 'Quantity

In [10]:
print(EUCS_LIST)

{'item40item52': 904324.1903397858, 'item40item60': 891137.6564243813, 'item40item58': 908332.6769529817, 'item40item9': 823674.2053906334, 'item52item60': 895507.5438753155, 'item52item58': 913114.3233783817, 'item52item9': 828247.933466712, 'item60item58': 899516.0304885112, 'item60item9': 816622.981774074, 'item58item9': 831109.463569841}


In [11]:
print(ORDER_LIST)

['item40', 'item52', 'item60', 'item58', 'item9']


## Searching stage

In [12]:
def construct(prefix_list: list, x: list, y: list) -> list:
    pre_pointer = 0
    x_pointer = 0
    y_pointer = 0
    result = list()

    while x_pointer < len(x) and y_pointer < len(y):
        if x[x_pointer][0] < y[y_pointer][0]:
            x_pointer += 1
        elif x[x_pointer][0] > y[y_pointer][0]:
            y_pointer += 1
        else:
            if prefix_list != None:
                while prefix_list[pre_pointer][0] < x[x_pointer][0]:
                    pre_pointer += 1
                result.append([pre_pointer, x[x_pointer][1] + y[y_pointer][1] - prefix_list[pre_pointer][1], min(x[x_pointer][2], y[y_pointer][2])])
                x_pointer += 1
                y_pointer += 1
                pre_pointer += 1
            else:
                result.append([x_pointer, x[x_pointer][1] + y[y_pointer][1], min(x[x_pointer][2], y[y_pointer][2])])
                x_pointer += 1
                y_pointer += 1
    return result

def utility_condition(x, minutil):
    return sum(i[1] for i in x) >= minutil

def get_utility(x):
    return sum(i[1] for i in x)

def calculate_periodic(x):
    periodic = [x[0][0] + 1]
    keys = x
    for i in range(len(keys) - 1):
        periodic.append(keys[i+1][0] - keys[i][0])
    return max(periodic), min(periodic), sum(periodic) / len(periodic)

def periodic_condition(x, minper, maxper, minavg, maxavg):
    max_i, min_i, avg_i = calculate_periodic(x)
    return min_i >= minper and max_i <= maxper and avg_i >= minavg and avg_i <= maxavg

def ru_condition(x, minutil):
    return sum(i[2] for i in x) >= minutil

def max_per_condition(x, maxper):
    max_x, min_x, avg_x = calculate_periodic(x)
    return max_x <= maxper

def avg_per_condition(x, maxavg):
    max_x, min_x, avg_x = calculate_periodic(x)
    return avg_x <= maxavg

def get_name(X, Y):
    temp_x = X.split("item")
    temp_y = Y.split("item")
    return "item" + temp_x[-1] + "item" + temp_y[-1]

def eucp_condition(X, Y, eucs_list, minutil):
    items_name = get_name(X, Y)
    return eucs_list[items_name] >= minutil

# def is_mlist(Y):
#     return True if isinstance(Y[0], str) and isinstance(Y[1], str) else False

# def du_compute(Y, X):
#     if is_mlist(Y):
#         return sum(x[2][-1] + x[2][1] for x in Y) + sum(x[1] for x in X)
#     else:
#         return sum(x[-1] + x[1] for x in Y) + sum(x[1] for x in X)
    
# def gen_mlist_1(X, Y_MLIST, DU):
#     mlist = Y_MLIST
#     for i in mlist
    
#     return None

# def gen_mlist_2(X, Y_LIST, P, DU):
#     return None


In [13]:
def phm_seach(prefix : str, prefix_list: list, lists: dict, order_list: list) -> set:
    result = list()
    key_lists = list()

    for i in order_list:
      if i in lists:
        key_lists.append(i)

    # print(prefix, key_lists, order_list)
    for i in key_lists:
        # print(i)
        # print(i, lists[i])
        # print(lists[i], end='\t')
        # print(i, utility_condition(lists[i], EPM['min_util']), periodic_condition(lists[i], EPM['min_per'], EPM['max_per'], EPM['min_avg'], EPM['max_avg'])\
        #       , calculate_periodic(lists[i]))
        if utility_condition(lists[i], EPM['min_util']) and periodic_condition(lists[i], EPM['min_per'], EPM['max_per'], EPM['min_avg'], EPM['max_avg']):
            result.append((i, get_utility(lists[i])))
            if len(result) > EPM["top_k"]:
              result.sort(key=lambda x: x[1], reverse=True)
              result = result[:EPM["top_k"]]
              EPM["min_util"] = result[-1][1]

        # print(lists[i])
        # print(EPM)
        if ru_condition(lists[i], EPM['min_util']) and max_per_condition(lists[i], EPM['max_per']) and avg_per_condition(lists[i], EPM['max_avg']):
            new_list = dict()
            new_order_list = list()
            for j in key_lists[key_lists.index(i) + 1:]:
                if get_name(i, j) not in EUCS_LIST:
                  continue
                if eucp_condition(i, j, EUCS_LIST, EPM['min_util']):
                    
                    z = get_name(i, j) if prefix == None else prefix + get_name(i, j)

                    z_list = construct(prefix_list, lists[i], lists[j])

                    new_list[z] = z_list
                    new_order_list.append(z)
            result.extend(phm_seach(i, lists[i], new_list, new_order_list))
            if len(result) > EPM["top_k"]:
              result.sort(key=lambda x: x[1], reverse=True)
              result = result[:EPM["top_k"]]
              EPM["min_util"] = result[-1][1]
    return result

In [14]:
print(UNIT_UTILITY)

for i in NEW_DATASET:
    print(i)

{'item1': 5.569802276812463, 'item3': -44.26347305389221, 'item5': 16.58330528441602, 'item7': -5.504876462938881, 'item9': 16.416492693110648, 'item11': 10.955378111789573, 'item13': 21.75377468060395, 'item15': 26.902764067127343, 'item17': -16.7328, 'item19': 16.416666666666668, 'item21': 27.053932584269663, 'item23': -16.31260319207485, 'item25': 5.482867132867133, 'item27': 16.666666666666668, 'item29': -10.762024520591009, 'item31': -27.361441013460016, 'item34': 16.232565789473686, 'item36': -33.03194578896418, 'item38': -32.25136612021858, 'item40': -38.31230283911672, 'item42': 11.024318349299927, 'item44': -21.807044410413475, 'item46': 33.01643192488263, 'item48': -16.54231662794557, 'item50': 37.987848101265826, 'item52': -11.184929356357928, 'item54': 22.153429602888085, 'item56': -33.67030784508441, 'item58': -5.495148669796557, 'item60': -10.956494125119086, 'item62': -16.673529411764704, 'item64': -27.206385404789053, 'item66': -16.473684210526315, 'item68': 5.530745967

In [15]:
utility_list = dict()
pnu_utility_list = dict()
for i in NEW_DATASET:
    total = 0
    for j in range(len(i['Item']) - 1, -1, -1):

        if i['Item'][j] not in utility_list:
            utility_list[i['Item'][j]] = list()
            pnu_utility_list[i['Item'][j]] = list()

        utility_list[i['Item'][j]].append([int(i['Tid'][1:]), UNIT_UTILITY[i['Item'][j]]*i['Quantity'][j], total])
        
        if UNIT_UTILITY[i['Item'][j]] < 0:
            pnu_utility_list[i['Item'][j]].append([int(i['Tid'][1:]), 0, UNIT_UTILITY[i['Item'][j]]*i['Quantity'][j], total])
        else:
            pnu_utility_list[i['Item'][j]].append([int(i['Tid'][1:]), UNIT_UTILITY[i['Item'][j]]*i['Quantity'][j], 0, total])

        total += UNIT_UTILITY[i['Item'][j]]*i['Quantity'][j] if UNIT_UTILITY[i['Item'][j]] > 0 else 0
for i in utility_list.keys():
    print(i, utility_list[i])

item9 [[0, 16.416492693110648, 0], [1, 16.416492693110648, 0], [2, 16.416492693110648, 0], [3, 16.416492693110648, 0], [4, 16.416492693110648, 0], [5, 16.416492693110648, 0], [6, 16.416492693110648, 0], [7, 16.416492693110648, 0], [8, 16.416492693110648, 0], [9, 16.416492693110648, 0], [10, 16.416492693110648, 0], [11, 16.416492693110648, 0], [12, 16.416492693110648, 0], [13, 16.416492693110648, 0], [14, 16.416492693110648, 0], [15, 16.416492693110648, 0], [16, 16.416492693110648, 0], [17, 16.416492693110648, 0], [18, 16.416492693110648, 0], [19, 16.416492693110648, 0], [20, 16.416492693110648, 0], [21, 16.416492693110648, 0], [22, 16.416492693110648, 0], [23, 16.416492693110648, 0], [24, 16.416492693110648, 0], [25, 16.416492693110648, 0], [26, 16.416492693110648, 0], [27, 16.416492693110648, 0], [28, 16.416492693110648, 0], [29, 16.416492693110648, 0], [30, 16.416492693110648, 0], [31, 16.416492693110648, 0], [32, 16.416492693110648, 0], [33, 16.416492693110648, 0], [34, 16.416492693

In [16]:
for i, j in pnu_utility_list.items():
    print(i, j)

item9 [[0, 16.416492693110648, 0, 0], [1, 16.416492693110648, 0, 0], [2, 16.416492693110648, 0, 0], [3, 16.416492693110648, 0, 0], [4, 16.416492693110648, 0, 0], [5, 16.416492693110648, 0, 0], [6, 16.416492693110648, 0, 0], [7, 16.416492693110648, 0, 0], [8, 16.416492693110648, 0, 0], [9, 16.416492693110648, 0, 0], [10, 16.416492693110648, 0, 0], [11, 16.416492693110648, 0, 0], [12, 16.416492693110648, 0, 0], [13, 16.416492693110648, 0, 0], [14, 16.416492693110648, 0, 0], [15, 16.416492693110648, 0, 0], [16, 16.416492693110648, 0, 0], [17, 16.416492693110648, 0, 0], [18, 16.416492693110648, 0, 0], [19, 16.416492693110648, 0, 0], [20, 16.416492693110648, 0, 0], [21, 16.416492693110648, 0, 0], [22, 16.416492693110648, 0, 0], [23, 16.416492693110648, 0, 0], [24, 16.416492693110648, 0, 0], [25, 16.416492693110648, 0, 0], [26, 16.416492693110648, 0, 0], [27, 16.416492693110648, 0, 0], [28, 16.416492693110648, 0, 0], [29, 16.416492693110648, 0, 0], [30, 16.416492693110648, 0, 0], [31, 16.416

In [17]:
result = phm_seach(None, None, utility_list, ORDER_LIST)

In [18]:
print(result)

[('item9', 47181.0), ('item58item9', 31377.021378981386), ('item60item9', 15479.095940256078), ('item52item9', 14993.660523133294), ('item60item58item9', -99.61538851535427), ('item52item58item9', -755.1719791705979), ('item52item60item9', -16184.379338711708), ('item58', -17557.0), ('item52item60item58item9', -31719.16462822658), ('item60', -34502.0)]
